In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pandarallel import pandarallel
from tqdm.auto import tqdm
import warnings
from sklearn.metrics import average_precision_score

warnings.simplefilter(action='ignore', category=FutureWarning)
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

plt.rcParams["figure.figsize"] = (8, 4)


EXP_DIR = "../_EXPERIMENTS/"
EXP_NAME = "12folds__transformer_crop120_cutmix_1"

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [33]:
class_weights = np.load("../data/bce_class_weights.npz")["weights"]

In [35]:
class_weights.shape

(256,)

In [2]:
def process_prediction(pred):
    return np.array(list(map(float, pred.split(","))))


def unprocess_prediction(pred):
    pred = [p for p in pred]
    return ",".join(map(str, pred))


def process_tags(tags): 
    tags = list(map(int, tags.split(",")))
    one_hot = np.zeros(256)
    one_hot[tags] = 1
    return one_hot

In [3]:
features_cols = [f"f{i}" for i in range(256)]
target_cols = [f"tag{i}" for i in range(256)]

df = pd.read_csv(f"{EXP_DIR}/{EXP_NAME}/train_prediction.csv")
df.tags = pd.read_csv("../data/train.csv").tags
df[features_cols] = np.stack(df.prediction.parallel_apply(process_prediction))
df[target_cols] = np.stack(df.tags.parallel_apply(process_tags))
df = df[features_cols + target_cols]

/tmp/ipykernel_786390/1309276267.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.tags = pd.read_csv("../data/train.csv").tags


/tmp/ipykernel_786390/1309276267.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[features_cols] = np.stack(df.prediction.parallel_apply(process_prediction))
/tmp/ipykernel_786390/1309276267.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[features_cols] = np.stack(df.prediction.parallel_apply(process_prediction))
/tmp/ipykernel_786390/1309276267.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

/tmp/ipykernel_786390/1309276267.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[target_cols] = np.stack(df.tags.parallel_apply(process_tags))
/tmp/ipykernel_786390/1309276267.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[target_cols] = np.stack(df.tags.parallel_apply(process_tags))
/tmp/ipykernel_786390/1309276267.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    df[features_cols], 
    df[target_cols],
    test_size=0.1, 
    random_state=42,
)

# Model

In [13]:
from catboost import CatBoostClassifier, Pool, MultiTargetCustomMetric
from catboost.utils import eval_metric

train_pool = Pool(x_train, y_train)
test_pool = Pool(x_val, y_val)

In [25]:
class APMetric(MultiTargetCustomMetric):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return True

    def evaluate(self, approxes, target, weight):
        # assert len(approxes) == 1, f"{np.array(approxes).shape}  {np.array(target).shape}"
        # assert len(target) == len(approxes[0])
        preds = np.array(approxes).T
        target = np.array(target).T
        return average_precision_score(target, preds), 0


In [47]:
clf = CatBoostClassifier(
    loss_function='MultiLogloss',
    # eval_metric=APMetric(),#'HammingLoss',
    iterations=500,
    class_names=target_cols,
    # auto_class_weights="SqrtBalanced",
    task_type="GPU",
    devices=[0],
    # class_weights=class_weights,
)
clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.068517
0:	learn: 0.5127237	test: 0.5127887	best: 0.5127887 (0)	total: 2.91s	remaining: 24m 12s
50:	learn: 0.0416279	test: 0.0431188	best: 0.0431188 (50)	total: 2m 4s	remaining: 18m 14s
100:	learn: 0.0375719	test: 0.0397798	best: 0.0397798 (100)	total: 3m 53s	remaining: 15m 20s
150:	learn: 0.0354778	test: 0.0382500	best: 0.0382500 (150)	total: 5m 50s	remaining: 13m 29s
200:	learn: 0.0339091	test: 0.0372370	best: 0.0372370 (200)	total: 7m 51s	remaining: 11m 41s
250:	learn: 0.0325753	test: 0.0365130	best: 0.0365130 (250)	total: 9m 56s	remaining: 9m 51s
300:	learn: 0.0313549	test: 0.0359933	best: 0.0359933 (300)	total: 12m 2s	remaining: 7m 57s
350:	learn: 0.0302800	test: 0.0355987	best: 0.0355987 (350)	total: 14m 11s	remaining: 6m 1s
400:	learn: 0.0293078	test: 0.0353218	best: 0.0353218 (400)	total: 16m 19s	remaining: 4m 1s
450:	learn: 0.0284110	test: 0.0350992	best: 0.0350992 (450)	total: 18m 28s	remaining: 2m
499:	learn: 0.0276262	test: 0.0349301	best: 0.0349301 (4

In [32]:
val_pred = clf.predict_proba(x_val)
average_precision_score(y_val, x_val)

0.4999084785084441

In [48]:
val_pred = clf.predict_proba(x_val)
average_precision_score(y_val, val_pred)

0.414196455821642

# Prediction

In [52]:
test = pd.read_csv(f"{EXP_DIR}/{EXP_NAME}/prediction.csv")
x_test = test.copy()
x_test[features_cols] = np.stack(test.prediction.parallel_apply(process_prediction))
x_test = x_test[features_cols]

/tmp/ipykernel_786390/1215011428.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[features_cols] = np.stack(test.prediction.parallel_apply(process_prediction))
/tmp/ipykernel_786390/1215011428.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[features_cols] = np.stack(test.prediction.parallel_apply(process_prediction))
/tmp/ipykernel_786390/1215011428.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

In [53]:
prediction = clf.predict_proba(x_test)

In [54]:
test["prediction"] = [unprocess_prediction(p) for p in prediction]

In [56]:
test.to_csv(f"{EXP_DIR}/{EXP_NAME}/catboost_pred.csv", index=False)
clf.save_model(f"{EXP_DIR}/{EXP_NAME}/catboost_classifier")